<a href="https://colab.research.google.com/github/danyamrajuSanjay/Machine_Learning/blob/master/ImageDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-08-14 06:58:44--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  91.6MB/s    in 1.6s    

2020-08-14 06:58:45 (91.6 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [5]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [6]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

In [7]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

['horse46-1.png', 'horse08-3.png', 'horse05-0.png', 'horse40-2.png', 'horse18-3.png', 'horse35-8.png', 'horse38-5.png', 'horse01-2.png', 'horse03-7.png', 'horse44-6.png']
['human17-10.png', 'human17-11.png', 'human13-23.png', 'human09-18.png', 'human14-06.png', 'human10-21.png', 'human01-12.png', 'human13-16.png', 'human15-24.png', 'human05-20.png']
